In [1]:
'''
cuda: 1
'''
import os
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "4"
import math
from visdom import Visdom
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import numpy.linalg as la
import pandas as pd
from utils import *
from gen_data_nomix import *
from networks import GetFeature, D, TGCN
from graph_part.metis_1 import graph_part, get_part_data
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [2]:
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a,b)
    mape = mean_absolute_percentage_error(a,b)
    F_norm = 1-mae/a.mean()
    r2 = 1 - ((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, mape, F_norm, r2, var

def tgcn_loss(y_pred, y_true):
    lambda_loss = 0.0015
    Lreg = 0 # 正则化项
    for para in net.parameters():
        Lreg += torch.sum(para ** 2) / 2
    Lreg = lambda_loss * Lreg

    regress_loss = torch.sum((y_pred-y_true) ** 2) / 2
    return regress_loss + Lreg

In [3]:
# os.environ['CUDA_VISIBLE_DEVICES']='5'
device = torch.device('cuda:1')
cwd = os.getcwd()
viz = Visdom()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(1)
#===== para set: =====#
epochs = 500
load_batch_size = 32
seq_len = 12
pre_len = 6
gru_units = 64
src_city = 'bj'
tgt_city = 'sh'
dt_similar_lambda = 1
sub_graph_num = 4
min_mape = 100
if_norm = 'meanvalue'
inv_if_norm = 'inv_'+if_norm
tgt_num_days = 10
environ='TEST'+str(pre_len*5)+'min_'+str(tgt_num_days)+'DAY_main_400_nomix'

Setting up a new session...


In [4]:
#===== data load: =====# 
src_speed = CityDataset(city=src_city, typo='speed', seq_len=seq_len, pre_len=pre_len, num_days=tgt_num_days, if_norm=if_norm)
src_loader = DataLoader(src_speed, batch_size=load_batch_size, pin_memory=True, num_workers=0, shuffle=True)
tgt_speed = CityDataset(city=tgt_city, typo='speed', seq_len=seq_len, pre_len=pre_len, num_days=tgt_num_days, if_norm=if_norm)
tgt_loader = DataLoader(tgt_speed, batch_size=load_batch_size, pin_memory=True, num_workers=0, shuffle=True)
src_num_nodes, tgt_num_nodes = src_speed.data.shape[2], tgt_speed.data.shape[2]
src_adj = pd.read_csv('./src_data/'+src_city+'_adj.csv', header=None).values
tgt_adj = pd.read_csv('./src_data/'+tgt_city+'_adj.csv', header=None).values
tgt_adj = tgt_adj[:400, :400]
tgt_nodes_part, tgt_min_num = graph_part('sh',tgt_adj, sub_graph_num)
src_nodes_part, src_min_num = graph_part('bj',src_adj, (src_num_nodes//tgt_min_num)-1)

#===== model init: =====#
net = TGCN(tgt_min_num, 1, gru_units, seq_len, pre_len).to(device=device)
# net = net.double()
net.train()
set_requires_grad(net, requires_grad=True)

In [5]:
src_speed.__len__(), tgt_speed.__len__()

(8910, 2862)

In [6]:
# if os.path.exists(cwd+'/pkl/NETS_params.pkl'):
#     checkpoint = torch.load(cwd+'/pkl/NETS_params.pkl')
#     TranFeatureNet.load_state_dict(checkpoint['TF_model'])
#     discriminator.load_state_dict(checkpoint['D'])
#     net.load_state_dict(checkpoint['net'])
#     min_mape = checkpoint['min_mape']
#     print('加载模型成功')

#===== optim: =====# 
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)

Train_loss = []
Val_loss = []

validation_maes = []
validation_pred = []
validation_mapes = []
validation_acc = []
validation_r2 = []
validation_var = []
validation_rmses = []


viz.line([0.],[0.],env=environ, win='train_loss',opts=dict(title='train loss'))
viz.line([0.],[0.],env=environ, win='val_loss',opts=dict(title='val loss'))
viz.line([0.],[0.],env=environ, win='rmses',opts=dict(title='rmses'))
viz.line([0.],[0.],env=environ, win='maes',opts=dict(title='maes'))
viz.line([0.],[0.],env=environ, win='mapes',opts=dict(title='mapes'))

'mapes'

In [7]:
for epoch in range(epochs):
    torch.manual_seed(3)
#     batches = zip(src_loader)
    total_batch_loss = 0
    n_batches = len(src_loader)
    total_domain_loss = total_label_accuracy = 0
    total_similar_loss = 0
    tgcn_tlt_loss = []
    print('==========EPOCH %03d==========='%(epoch))
    # print('==========EPOCH {0:0>3}==========='.format(epoch))

##########################
# mix_data混合数据，输出tgt_data_gen。
##########################
    for src_data, src_pre in tqdm(src_loader, leave=False, total=n_batches):
        # ===== 数据转移到GPU
        src_data = src_data.to(device)
        if src_data.shape[0] != load_batch_size:
            continue
        

#         src_data = torch.cat((src_data1, src_data2, src_data3),0)
#         src_pre = torch.cat((src_pre1, src_pre2, src_pre3), 0)

        
    ### Training
        optimizer.zero_grad()
        tgcnloss = 0
                
        train_out_item = torch.zeros(src_data.size(0), pre_len, 0).to(device)
        train_y_item = torch.zeros(src_data.size(0), pre_len, 0).to(device)
        for i in range((src_num_nodes//tgt_min_num)-1):
            adj, speed, pre = get_part_data(i, src_nodes_part, src_adj, src_data, src_pre)
            adj = normalized_adj(adj)
            adj = adj[:tgt_min_num, :tgt_min_num]
            adj = adj.to(device=device,dtype=torch.float32)

            X_batch = speed[:,:,:tgt_min_num].permute(1,0,2)
            y_batch = pre[:,:,:tgt_min_num]
            X_batch = X_batch.to(device=device)
            y_batch = y_batch.to(device=device)

            h0 = torch.zeros(X_batch.size(1), tgt_min_num, gru_units, dtype=torch.float32).to(device=device)
            out = net(adj, X_batch, h0)


            train_out_item = torch.cat((train_out_item,out),2)
            train_y_item = torch.cat((train_y_item,y_batch),2)

#             loss = tgcn_loss(out, y_batch)
#             tgcnloss = tgcnloss + loss

#         tgcnloss = tgcnloss / load_batch_size
#         tgcn_tlt_loss.append(tgcnloss.item())

        tgcnloss=tgcn_loss(train_out_item,train_y_item) 
        tgcn_tlt_loss.append(tgcnloss.item()/load_batch_size)
        
        train_out_cpu = train_out_item.detach().cpu().numpy().reshape(-1,train_out_item.size(2))
        train_y_cpu = train_y_item.detach().cpu().numpy().reshape(-1,train_out_item.size(2))

        train_out_cpu = Normalize(train_out_cpu, src_speed.mean, src_speed.std, src_speed.max, src_speed.min, inv_if_norm)
        train_y_cpu = Normalize(train_y_cpu, src_speed.mean, src_speed.std, src_speed.max, src_speed.min, inv_if_norm)

        train_rmse, train_mae, train_mape, train_acc, train_r2_score, train_var_score = evaluation(train_y_cpu,train_out_cpu)
        
        # tqdm.write('TRAIN: rmse={0:.4f}, mae={1:.4f}, acc={2:.4f}, r2={3:.4f}, var={4:.4f}'.format(train_rmse, train_mae, train_acc, train_r2_score, train_var_score))
        tgcnloss.backward()
        optimizer.step()
        
        # total_batch_loss += batch_loss.item()
        # total_domain_loss += domain_loss.item()
        # total_similar_loss += similar_loss

    Train_loss.append(sum(tgcn_tlt_loss) / n_batches)
    viz.line([Train_loss[-1]],[epoch],env=environ, win='train_loss', update='append')
    # mean_domain_loss.append(total_domain_loss / n_batches)
    # mean_similar_loss.append(total_similar_loss / n_batches)
    # mean_total_batch_loss.append(total_batch_loss / n_batches)
    print('Trainning loss={0:.4f}'.format(Train_loss[-1]))

    with torch.no_grad():
        net.eval()
        tgt_val_speed = CityDataset(city=tgt_city, typo='speed', seq_len=seq_len, pre_len=pre_len, num_days=tgt_num_days, if_norm=if_norm)
        tgt_val_loader = DataLoader(tgt_val_speed, batch_size=tgt_val_speed.__len__(), pin_memory=True, num_workers=0, shuffle=True)
        for i ,(speed, pre) in enumerate(tgt_val_loader):
            tgt_dt = speed.to(device)
            tgt_pre = pre.to(device)
        
        out_item = torch.zeros(tgt_dt.size(0), pre_len, 0).to(device)
        y_item = torch.zeros(tgt_dt.size(0), pre_len, 0).to(device)
        
        for i in range(sub_graph_num):
            adj, speed, pre = get_part_data(i, tgt_nodes_part, tgt_adj, tgt_dt, tgt_pre)
            adj = normalized_adj(adj)
            adj = adj[:tgt_min_num, :tgt_min_num]
            adj = adj.to(device=device,dtype=torch.float32)

            X_batch = speed[:,:,:tgt_min_num].permute(1,0,2)
            y_batch = pre[:,:,:tgt_min_num]
            X_batch = X_batch.to(device=device)
            y_batch = y_batch.to(device=device)        

            h0 = torch.zeros(X_batch.size(1), tgt_min_num, gru_units, dtype=torch.float32).to(device=device)
            out = net(adj, X_batch, h0)

            out_item = torch.cat((out_item,out),2)
            y_item = torch.cat((y_item,y_batch),2)

#             loss = tgcn_loss(out, y_batch)
#             tgcnloss = tgcnloss + loss
            # optimizer.step()
        
        tgcnloss=tgcn_loss(out_item,y_item)
        tgcnloss = tgcnloss.item() / (tgt_dt.size(0))
        Val_loss.append(tgcnloss)
        out_cpu = out_item.detach().cpu().numpy().reshape(-1,out_item.size(2))
        y_cpu = y_item.detach().cpu().numpy().reshape(-1,out_item.size(2))

        out_cpu = Normalize(out_cpu, tgt_speed.mean, tgt_speed.std, tgt_speed.max, tgt_speed.min, inv_if_norm)
        y_cpu = Normalize(y_cpu, tgt_speed.mean, tgt_speed.std, tgt_speed.max, tgt_speed.min, inv_if_norm)

        rmse, mae, mape, acc, r2_score, var_score = evaluation(y_cpu,out_cpu)

        
        # validation_pred = []
        validation_maes.append(mae)
        validation_acc.append(acc)
        validation_mapes.append(mape)
        validation_r2.append(r2_score)
        validation_var.append(var_score)
        validation_rmses.append(rmse)
        
             
        viz.line([Val_loss[-1]],[epoch],env=environ, win='val_loss', update='append')
        viz.line([rmse],[epoch],env=environ, win='rmses', update='append')
        viz.line([mae],[epoch],env=environ, win='maes', update='append')
        viz.line([mape],[epoch],env=environ, win='mapes', update='append')
        
        print('VAL: rmse={0:.4f}, mae={1:.4f}, mape={2:.4f}, acc={3:.4f}, r2_score={4:.4f}, var_score={5:.4f}'.format(rmse, mae, mape, acc, r2_score, var_score))
        print('VAL loss={0:.4f}'.format(Val_loss[-1]))


#     if not os.path.exists(cwd+'/pkl'):
#         os.makedirs(cwd+'/pkl')
#     if validation_mapes[-1] < min_mape:
#         state = {'TF_model':TranFeatureNet.state_dict(),'D':discriminator.state_dict(), 'net':net.state_dict(), 'min_mape':validation_mape[-1]}
#         torch.save(state, cwd+'/pkl/NETS_params.pkl')
#         min_mape = validation_mapes[-1]

    # tqdm.write(f'EPOCH {epoch:03d}: mean_tgcn_tlt_loss={mean_tgcn_tlt_loss[-1]:.4f}, '
            #    f'domain_loss={mean_domain_loss[-1]:.4f}, similar_loss={mean_similar_loss[-1]:.4f}, '
            #    f'mean_total_batch_loss={mean_total_batch_loss[-1]:.4f}')   

    # tqdm.write(f'EPOCH {epoch:03d}: mean_tgcn_tlt_loss={mean_tgcn_tlt_loss[-1]:.4f}, '
            #    f'rmse={rmse:.4f}, mae={mae:.4f}, acc={acc:.4f}, r2={r2_score:.4f}, var={var_score:.4f}')

np.savez('losses_400',Train_loss=Train_loss, Val_loss=Val_loss, Val_maes=validation_maes, Val_mape=validation_mapes, Val_acc=validation_acc, \
                Val_rmse=validation_rmses, Val_r2=validation_r2, Val_var=validation_var)

  0%|          | 1/279 [00:00<00:51,  5.44it/s]

==========EPOCH 000===========


Trainning loss=58.2180


  0%|          | 1/279 [00:00<00:34,  7.96it/s]

VAL: rmse=6.4795, mae=4.5529, mape=0.1421, acc=0.8707, r2_score=0.6592, var_score=0.6611
VAL loss=1.5210
==========EPOCH 001===========


...

VAL: rmse=9.0223, mae=5.4011, mape=0.1728, acc=0.8466, r2_score=0.3393, var_score=0.3466
VAL loss=2.9495
==========EPOCH 498===========


Trainning loss=2.3608


  0%|          | 1/279 [00:00<00:38,  7.30it/s]

VAL: rmse=8.9496, mae=5.3834, mape=0.1722, acc=0.8471, r2_score=0.3499, var_score=0.3566
VAL loss=2.9022
==========EPOCH 499===========


Trainning loss=2.3599
VAL: rmse=9.2326, mae=5.4429, mape=0.1746, acc=0.8454, r2_score=0.3081, var_score=0.3165
VAL loss=3.0886
